# Sync activity video clips with accelerometer data

Project status:
- COMPLETE: Get start times for videos using Python
- COMPLETE: Use start/stop frame number and convert to UTC
- IN PROGRESS: deal with cisrol12
- Modify GUI function to use my start/stop times to label data for relevant subjects and cycles below

Notes:
- fps = 29.97 aka Video Frame Rate
- 33.367 milliseconds per frame

In [1]:
# Importing the Libraries
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import datetime as dt
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

## RTO drive is X:

## test case video clip to get timestamps


In [9]:
# subject 1050
id = 'cisuabn14'
path = r'X:\CIS-PD Videos'
subj_path = os.path.join(path,id)
video_name = 'cisuabn14_cycle2.mp4'
video_clip_path = os.path.join(subj_path,video_name)

In [10]:
video_clip_path

'X:\\CIS-PD Videos\\cisuabn14\\cisuabn14_cycle2.mp4'

In [11]:
# these give my downloaded time not the actual time
import os.path, time
print("Last modified: %s" % time.ctime(os.path.getmtime(video_clip_path)))
print("Created: %s" % time.ctime(os.path.getctime(video_clip_path)))

Last modified: Thu Jul 12 08:55:07 2018
Created: Thu Jul 12 09:04:26 2018


# Load all sec_annotation.csv files for each subj, concatenate into 1 df

In [2]:
# read in timestamp file with mp4 metadata
path = r'X:\CIS-PD Videos\timestamp'
filename = os.path.join(path, 'video_utc_timestamp.csv')
timestamp_df = pd.read_csv(filename)
timestamp_df = timestamp_df.drop(columns=['Unnamed: 0','videoname','CreateDate','ModifyDate','UTC_modify_date'])

In [3]:
# list of subjects without ciscid4, ciscih8, ciccij10 due to it being edited multiple times
# omitted cisrol12 as it doesn't have a sec_annotation file
names_minus3 = ['cisnwh8','cisuabd4','cisuabe5','cisuabf6','cisuabg7','cisnwe5','cisnwf6','cisuabn14']

# create empty list
appended_data = []

# create 1 dataframe from each subject's sec_annotation.csv file
for i, k in enumerate(names_minus3):
    path = r'X:\CIS-PD Videos'
    path_subj = os.path.join(path,k) 
    path_file = os.path.join(path_subj,'sec_annotation.csv')
    data = pd.read_csv(path_file)
    appended_data.append(data)
    
# concatenate list of dataframes
appended_data = pd.concat(appended_data, ignore_index=True)
appended_data = appended_data.drop(columns=['Unnamed: 0'])

# combine subjid and cycle number column to create a key for merge
# combine strings of both columns
timestamp_df.cycle = timestamp_df.cycle.astype(str)
timestamp_df.cycle = timestamp_df.subjid + timestamp_df.cycle
# drop subjid column
timestamp_df = timestamp_df.drop(columns=['subjid'])
# change name of column
timestamp_df = timestamp_df.rename(index=str,columns={'cycle':'subj_cycle'})

In [4]:
# Combine subject code and cycle column to create a key for merge in appended_data dataframe that
# has the activity clip frame annotations
appended_data.cycle = appended_data.cycle.astype(str)
appended_data['subj_cycle'] = appended_data['subject code'] + appended_data.cycle

# Merge dataframes based on subj_cycle columns in both

In [5]:
utc_df = pd.merge(timestamp_df, appended_data, on='subj_cycle',how='outer')

# Transform start and stop frame with UTC create time

In [6]:
utc_df['start_utc'] = utc_df['start frame']*33.367+utc_df.UTC_create_date
utc_df['stop_utc'] = utc_df['stop frame']*33.367+utc_df.UTC_create_date

# Adjust UAB site data by... 1yr 5 hrs
- UAB subject: 1003, 1005, 1007, 1009, 1050
- cisuabd4 cisuabe5 cisuabf6 cisuabg7 cisuabn14

In [7]:
# millisecond conversions
year = 31556952000
fivehr = 18000000
uab_convertor = year + fivehr

In [8]:
# Add 1 year and 5 hrs to uab subjects
uab_names = ('cisuabd4','cisuabe5','cisuabf6','cisuabg7','cisuabn14')
for i, k in enumerate(uab_names):
    utc_df.loc[utc_df['subject code'] == k, 'start_utc'] += uab_convertor
    utc_df.loc[utc_df['subject code'] == k, 'stop_utc'] += uab_convertor

# Combine NtsBts activity split into cycle 6 part 1 and 2 videos into 1 row

In [9]:
# combine rows 503 and 504
utc_df.stop_utc[503] = utc_df.stop_utc[504]

C:\ProgramData\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
# drop row and reindex
utc_df = utc_df.drop([504]).reset_index(drop=True)

# Change cycle 7 to 6

In [11]:
utc_df.loc[utc_df.cycle == '7', 'cycle'] = '6'

# temp dataframe

In [12]:
df = utc_df.copy()

# Deal with cisrol12 (1048) separately
- utc_df does NOT have cisrol12 data on it
- Create separate script for cisrol12 since it doesn't have sec_annotation.csv

In [13]:
def keeprightstring(string, sep='cisrol12'):
    """Take a string and keep text after specified character.
    Default character is 'cisrol12'."""
    new_string = string.split(sep, 1)[-1]
    return new_string

In [14]:
# Add necessary data for cisrol12
# subject code
df.loc[df['subj_cycle'].str.contains('cisrol12'), 'subject code'] = 'cisrol12'
# start frame
df.loc[df['subj_cycle'].str.contains('cisrol12'), 'start frame'] = 0
# start_utc = create time
df.loc[df['subj_cycle'].str.contains('cisrol12'), 'start_utc'] = df.UTC_create_date
# cycle
df.loc[df['subj_cycle'].str.contains('cisrol12'), 'cycle'] = df.subj_cycle
for i in range(391,433):
    df.cycle[i] = keeprightstring(df.cycle[i])
# SKIP start time
# SKIP stop time

C:\ProgramData\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [15]:
df.loc[df['subj_cycle'].str.contains('cisrol12')].head(5)

,UTC_create_date,subj_cycle,subject code,start frame,stop frame,activity,cycle,shortname,start time sec,stop time sec,start_utc,stop_utc
391,1505134245000,cisrol121,cisrol12,0.0,NaN,NaN,1,NaN,NaN,NaN,1.505134e+12,NaN
392,1505134286000,cisrol121,cisrol12,0.0,NaN,NaN,1,NaN,NaN,NaN,1.505134e+12,NaN
393,1505134361000,cisrol121,cisrol12,0.0,NaN,NaN,1,NaN,NaN,NaN,1.505134e+12,NaN
394,1505137757000,cisrol122,cisrol12,0.0,NaN,NaN,2,NaN,NaN,NaN,1.505138e+12,NaN
395,1505137780000,cisrol122,cisrol12,0.0,NaN,NaN,2,NaN,NaN,NaN,1.505138e+12,NaN


# can remove next 2 lines

In [16]:
df['subj_cycle'][391] # - 'cisrol12'

'cisrol121'

In [17]:
df['subj_cycle'][391].replace('cisrol12','')

'1'

# Add column containing 4 digit id
- need to complete cisrol12 data first to add its 4 digit id
- make sure to user 'df' dataframe

In [18]:
# Get subject id or code
path_id = r'X:\CIS-PD MUSC\decoded_forms'
filename_id = os.path.join(path_id, 'videoID.csv') # ie. file = 'videoID.csv'
subjid_df = pd.read_csv(filename_id)
subjid_df.SubjectCode = subjid_df.SubjectCode.astype('int')
# get 4 digit subject code
reverse_id_dict = subjid_df.set_index('FoxInsightID').to_dict()['SubjectCode']

In [19]:
# test
reverse_id_dict.get('cisrol12', 'Unknown')

1048

In [20]:
df['subject_number'] = df['subject code']
for i, k in enumerate(df.subject_number):
    df['subject_number'][i] = reverse_id_dict.get(k,1048)

C:\ProgramData\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
df.groupby('subject_number').count()
# unknown is cisrol12

,UTC_create_date,subj_cycle,subject code,start frame,stop frame,activity,cycle,shortname,start time sec,stop time sec,start_utc,stop_utc
subject_number,,,,,,,,,,,,
1003,90,90,90,90,90,90,90,90,90,90,90,90
1005,60,60,60,60,60,60,60,60,60,60,60,60
1007,60,60,60,60,60,60,60,60,60,60,60,60
1009,45,45,45,45,45,45,45,45,45,45,45,45
1019,15,15,15,15,15,15,15,15,15,15,15,15
1024,15,15,15,15,15,15,15,15,15,15,15,15
1030,90,90,90,90,90,90,90,90,90,90,90,90
1048,58,58,42,42,0,0,42,0,0,0,42,0
1050,75,75,75,75,75,75,75,75,75,75,75,75


In [22]:
df.head(5)

,UTC_create_date,subj_cycle,subject code,start frame,stop frame,activity,cycle,shortname,start time sec,stop time sec,start_utc,stop_utc,subject_number
0,1501681699000,cisnwh81,cisnwh8,1118.0,2145.0,Standing,1,Stndg,37.0,71.0,1.501682e+12,1.501682e+12,1030
1,1501681699000,cisnwh81,cisnwh8,2315.0,3333.0,Walking,1,Wlkg,77.0,111.0,1.501682e+12,1.501682e+12,1030
2,1501681699000,cisnwh81,cisnwh8,3608.0,4800.0,Walking while counting,1,WlkgCnt,120.0,160.0,1.501682e+12,1.501682e+12,1030
3,1501681699000,cisnwh81,cisnwh8,5832.0,6518.0,Finger to nose--right hand,1,FtnR,194.0,217.0,1.501682e+12,1.501682e+12,1030
4,1501681699000,cisnwh81,cisnwh8,6518.0,7121.0,Finger to nose--left hand,1,FtnL,217.0,237.0,1.501682e+12,1.501682e+12,1030


# Adjust cycle for Nick's GUI

In [23]:
# Change cycle dtype from str to integer
df.cycle = pd.to_numeric(df.cycle, downcast='integer')

In [24]:
# Start cycle 1 at 0 for GUI
df.cycle += -1

# Save utc_df as csv file

In [30]:
# datat set has missing data for ciscid4, ciscih8, ciscij10, and cisro12
path = r'X:\CIS-PD Videos\timestamp'
fname = 'GUI_timestamp_missing4.csv'
filename = os.path.join(path, fname)
with open(filename,'wb') as f:
    df.to_csv(filename, sep=',')

In [64]:
df.shape

(508, 13)

# Drop NaN for 4 subjects

In [39]:
df_minus3 = df.dropna()

In [40]:
print(df_minus3.shape)

(450, 13)


In [45]:
df_minus3 = df_minus3.reset_index(drop=True)

# Fix Drinking activity label with full name
'Taking a glass of water and drinking'

In [54]:
df_minus3.loc[df_minus3['shortname'].str.contains('Drnkg'), 'activity'] = 'Taking a glass of water and drinking'

In [ ]:
# check label
df_minus3.loc[df_minus3['shortname'].str.contains('Drnkg')]

# Save utc_df as csv file

In [57]:
# datat set has missing data for ciscid4, ciscih8, ciscij10, and cisro12
path = r'X:\CIS-PD Videos\timestamp'
fname = 'GUI_timestamp.csv'
filename = os.path.join(path, fname)
with open(filename,'wb') as f:
    df_minus3.to_csv(filename, sep=',')

Project status

Completed
1019

In progress
1003 (cycle 0-5 / time 0,30,60,90,120,150) offset -21800
- Error: saved Standing, Walking, Walk with count to cycle 2. Can I just save over in this situation?
- Not clear on stand, walk, and walk with count in cycle 1
1005 (cycle 0-3 / time 0,30,60,90) offset -20420
- Error: saved first cycle under '2 Weeks: Time 90' with offset -20929.32926742833
1007 (cycle 0-3/ time 0,30,60,90) offset -21940.197417575815
- unclear on activities
1009 (cycle 0,3,5 / time 0,90,150)
- offset -19000 doesn't look right
1023 - skip UAB
1024 (cycle 2 / time 60)
- no offset, but walk vs walk and count not very clear
1038 ???
1039 - skip UAB
1043 - skip UAB
1048 - skip cisrol12
1050 (cycle 1-5 / time 30,60,90,120,150) offset -20800
- unclear on activities


Steps to complete annotation:
Load GUI
Set subj number
Set time: 90
Activity: Standing
Look for standing and walking activities
Set offset
Check start/stop, adjust
Save Data
Next Task - don't modify NtsBts activity

Note on cycle/time
cycle 0 / time 0
cycle 1 / time 30
cycle 2 / time 60
cycle 3 / time 90
cycle 4 / time 120

Summary of data that is off
- 1003 is off by 1 yr and 5 hours and some seconds, cycle 3 was off by an additional 9 min
- 1030 is off by several seconds (usually around 30 sec)
- 1005 is off by 1 yr 5 hrs and some seconds, cycle 2 is off by an additional 5 min
- 1007 is off by 1 yr 5 hrs and some seconds
- 1009 is off by 1 yr 5 hrs and some seconds
- 1019 is off by 29.5 min
- 1024 is off by 50 sec
- 1048 cycle 1 3.5 hours off, cycle 2 missing, cycle 4 is 4 hrs off, cycle 5 is about 4 hrs off
- 1050 is off by 1 yr, 5 hrs and some sec

These videos we suspect editing, so Skip these subjects (ciscid4, ciscih8, ciccij10):
- 1023 is off by 18 days, but the watch shaking time for all cycles the same
- 1039 is off by 2 months, 13 days, and variable time but the watch shaking time for all cycles the same
- 1043 is off by 2 months, 1 day, but  the watch shaking time for all cycles the same

# missing data in GUI_timestamp_missing4 for ciscid4, ciscih8, ciscij10
# and cisrol12

# correct subject_number

In [72]:
# ciscid4
df.loc[df['subj_cycle'].str.contains('ciscid4'), 'subject_number'] = 1023
# ciscih8
df.loc[df['subj_cycle'].str.contains('ciscih8'), 'subject_number'] = 1039
# ciscij10
df.loc[df['subj_cycle'].str.contains('ciscij10'), 'subject_number'] = 1043

# for cisrol12, add the following data
################### work on this
# stop frame
# stop_utc = ?
# activity
# shortname

# Add cisrol12 missing data

In [108]:
# missing data

# shortname list
shortname = (
# cisrol12 cycle 0 (cycle 1 folder)
'Fldg_trial1', 'Fldg_trial2', 'Sitng',
# cisrol12 cycle 1 (cycle 2 folder)
'none','none',
'Stndg', 'Wlkg', 'WlkgCnt', 'FtnR', 'FtnL', 
'RamR', 'RamL', 'SitStand', 'Drwg', 'Typg', 'NtsBts',
'none',
'Drnkg', 'Sheets_trial1', 'Sheets_trial2',
'Fldg', 'Sitng',
# cycle 2 empty (cycle 3) not included
# cisrol12 cycle 3 (cycle 4 folder)
'none',
'Stndg', 'FtnR', 'FtnL', 'RamR', 'RamL', 
'SitStand', 'Drwg', 'Typg', 'NtsBts','Drnkg','Sitng',
# cisrol12 cycle 4 (cycle 5 folder)
'FtnL', 'RamR', 'RamL', 'SitStand', 'Drwg', 'Typg', 'NtsBts', 'Sitng'
)

# end frame
stopframe = (585,1288,1033,
0, 0,# shaking for video 35 and 36
1048,1063,988,988,1228,1333,538,403,1438,988,973,
253, # nothing for video 50
1363,1048,1258,2158,1048,
# omitted for cycle 3
0, # shaking for 75
1453,913,958,1198,523,403,988,973,1093,1348,1258,
958,703,583,403,988,943,958,1003)

# add shortname
df.loc[df['subject code']=='cisrol12', 'shortname'] = shortname
# add end frame
df.loc[df['subject code']=='cisrol12', 'stop frame'] = stopframe


# create stop_utc column which will add in cisrol12 data

In [112]:
df['stop_utc'] = df['stop frame']*33.367+df.UTC_create_date

In [114]:
df.loc[df['subject code']=='cisrol12'].shape

(42, 13)

# Delete 'none' Shortnames (should be 4 rows) to get rid of Shaking rows

In [124]:
df.drop(df[df.shortname == 'none'].index, inplace=True)

In [125]:
df.loc[df['subject code']=='cisrol12'].shape

(38, 13)

# Next actions:

# combine Fldg_trial1 and Fldg_trial2 videos and timestamps
# combine Sheets_trial1 and Sheets_trial2 videos and timestamps
# Create activity name (full)


In [117]:
df.loc[df['shortname']=='none']

,UTC_create_date,subj_cycle,subject code,start frame,stop frame,activity,cycle,shortname,start time sec,stop time sec,start_utc,stop_utc,subject_number
394,1505137757000,cisrol122,cisrol12,0.0,0.0,NaN,1.0,none,NaN,NaN,1.505138e+12,1.505138e+12,1048
395,1505137780000,cisrol122,cisrol12,0.0,0.0,NaN,1.0,none,NaN,NaN,1.505138e+12,1.505138e+12,1048
407,1505139234000,cisrol122,cisrol12,0.0,253.0,NaN,1.0,none,NaN,NaN,1.505139e+12,1.505139e+12,1048
413,1505142200000,cisrol124,cisrol12,0.0,0.0,NaN,3.0,none,NaN,NaN,1.505142e+12,1.505142e+12,1048


In [121]:
df.drop(df[df.shortname == 'none'].index, inplace=True)

In [126]:
# cisrol data
df.loc[df['subject code']=='cisrol12']

,UTC_create_date,subj_cycle,subject code,start frame,stop frame,activity,cycle,shortname,start time sec,stop time sec,start_utc,stop_utc,subject_number
391,1505134245000,cisrol121,cisrol12,0.0,585.0,NaN,0.0,Fldg_trial1,NaN,NaN,1.505134e+12,1.505134e+12,1048
392,1505134286000,cisrol121,cisrol12,0.0,1288.0,NaN,0.0,Fldg_trial2,NaN,NaN,1.505134e+12,1.505134e+12,1048
393,1505134361000,cisrol121,cisrol12,0.0,1033.0,NaN,0.0,Sitng,NaN,NaN,1.505134e+12,1.505134e+12,1048
396,1505137835000,cisrol122,cisrol12,0.0,1048.0,NaN,1.0,Stndg,NaN,NaN,1.505138e+12,1.505138e+12,1048
397,1505137956000,cisrol122,cisrol12,0.0,1063.0,NaN,1.0,Wlkg,NaN,NaN,1.505138e+12,1.505138e+12,1048
398,1505138004000,cisrol122,cisrol12,0.0,988.0,NaN,1.0,WlkgCnt,NaN,NaN,1.505138e+12,1.505138e+12,1048
399,1505138430000,cisrol122,cisrol12,0.0,988.0,NaN,1.0,FtnR,NaN,NaN,1.505138e+12,1.505138e+12,1048
400,1505138485000,cisrol122,cisrol12,0.0,1228.0,NaN,1.0,FtnL,NaN,NaN,1.505138e+12,1.505139e+12,1048
401,1505138598000,cisrol122,cisrol12,0.0,1333.0,NaN,1.0,RamR,NaN,NaN,1.505139e+12,1.505139e+12,1048
402,1505138703000,cisrol122,cisrol12,0.0,538.0,NaN,1.0,RamL,NaN,NaN,1.505139e+12,1.505139e+12,1048


In [ ]:
# check after
df.loc[df['subject code']=='cisrol12'].head(3)

In [107]:
shortname = (
# cisrol12 cycle 0 (cycle 1 folder)
'Fldg_trial1', 'Fldg_trial2', 'Sitng',
# cisrol12 cycle 1 (cycle 2 folder)
'none','none',
'Stndg', 'Wlkg', 'WlkgCnt', 'FtnR', 'FtnL', 
'RamR', 'RamL', 'SitStand', 'Drwg', 'Typg', 'NtsBts',
'none',
'Drnkg', 'Sheets_trial1', 'Sheets_trial2',
'Fldg', 'Sitng',
# cycle 2 empty (cycle 3) not included
# cisrol12 cycle 3 (cycle 4 folder)
'none',
'Stndg', 'FtnR', 'FtnL', 'RamR', 'RamL', 
'SitStand', 'Drwg', 'Typg', 'NtsBts','Drnkg','Sitng',
# cisrol12 cycle 4 (cycle 5 folder)
'FtnL', 'RamR', 'RamL', 'SitStand', 'Drwg', 'Typg', 'NtsBts', 'Sitng'
)
print(len(shortname))

42


In [ ]:
cycle2_MVI_0035 shaking watch stand
cycle2_MVI_0036 shaking watch sit
cycle2_MVI_0037 Stndg
cycle2_MVI_0038 Wlkg
cycle2_MVI_0039 WlkgCnt
cycle2_MVI_0042 FtnR
cycle2_MVI_0043 FtnL
cycle2_MVI_0044 RamR
cycle2_MVI_0045 RamL
cycle2_MVI_0046 SitStand
cycle2_MVI_0047 Drwg
cycle2_MVI_0048 Typg
cycle2_MVI_0049 NtsBts
cycle2_MVI_0050 no activity
cycle2_MVI_0051 Drnkg
cycle2_MVI_0052 Sheets_trial1
cycle2_MVI_0053 Sheets_trial2
cycle2_MVI_0054 Fldg
cycle2_MVI_0055 Sitng

In [ ]:
# cisrol12 cycle 3 (cycle 4 folder)
('none','Stndg', 'FtnR', 'FtnL', 'RamR', 'RamL', 
'SitStand', 'Drwg', 'Typg', 'NtsBts','Drnkg','Sitng')
# end frame
()

### cycle4 video names:
- missing: Wlkg, WlkgCnt, Sheets, Fldg
- cycle4_MVI_0075   none
- cycle4_MVI_0076   Stndg
- cycle4_MVI_0078   FtnR
- cycle4_MVI_0079   FtnL
- cycle4_MVI_0080   RamR
- cycle4_MVI_0081   RamL
- cycle4_MVI_0082   SitStand
- cycle4_MVI_0083   Drwg
- cycle4_MVI_0084   Typg
- cycle4_MVI_0085   NtsBts
- cycle4_MVI_0086   Drnkg
- cycle4_MVI_0089   Sitng

In [ ]:
# cisrol12 cycle 4 (cycle 5 folder)
('FtnL', 'RamR', 'RamL', 'SitStand', 'Drwg', 'Typg', 'NtsBts', 'Sitng')
# end frame
()

### cycle5 video names:
- missing: Stndg, Wlkg, WlkgCnt, FtnR, Drnkg, Sheets, Fldg
- cycle5_MVI_0095   FtnL
- cycle5_MVI_0096   RamR
- cycle5_MVI_0097   RamL
- cycle5_MVI_0098   SitStand
- cycle5_MVI_0100   Drwg
- cycle5_MVI_0101   Typg
- cycle5_MVI_0102   NtsBts
- cycle5_MVI_0106   Sitng

# Save utc_df as csv file after cleaning cisrol12

In [30]:
# datat set has missing data for ciscid4, ciscih8, ciscij10
path = r'X:\CIS-PD Videos\timestamp'
fname = 'GUI_timestamp.csv'
filename = os.path.join(path, fname)
with open(filename,'wb') as f:
    df.to_csv(filename, sep=',')

In [64]:
df.shape

(508, 13)

# Subjects 1030, 1019, and 1024 UTC timestamps were not adjusted for any offset from watch timestamp.

# Combine cisuabn14 cycle 6 part 1 and 2 ntsbts video

In [ ]:
# option 1
ffmpeg -i input1.mp4 -c copy -bsf:v h264_mp4toannexb -f mpegts intermediate1.ts
ffmpeg -i input2.mp4 -c copy -bsf:v h264_mp4toannexb -f mpegts intermediate2.ts
ffmpeg -i "concat:intermediate1.ts|intermediate2.ts" -c copy -bsf:a aac_adtstoasc output.mp4

In [58]:
# option 2 - Worked
from moviepy.editor import VideoFileClip, concatenate_videoclips
path_video1 = r'//FS2.smpp.local\RTO\CIS-PD Videos\cisuabn14\cycle6_part1'
videoname1 = os.path.join(path_video1, 'NtsBts_part1.mp4')
path_video2 = r'//FS2.smpp.local\RTO\CIS-PD Videos\cisuabn14\cycle6_part2'
videoname2 = os.path.join(path_video2, 'NtsBts_part2.mp4')

clip1 = VideoFileClip(videoname1)
clip2 = VideoFileClip(videoname2)#.subclip(50,60)
final_clip = concatenate_videoclips([clip1,clip2])
path_video3 = r'//FS2.smpp.local\RTO\CIS-PD Videos\cisuabn14\cycle6_part1'
videoname_final = os.path.join(path_video3, 'NtsBts.mp4')
final_clip.write_videofile(videoname_final)

Imageio: 'ffmpeg-win32-v3.2.4.exe' was not found on your computer; downloading it now.
Try 1. Download from https://github.com/imageio/imageio-binaries/raw/master/ffmpeg/ffmpeg-win32-v3.2.4.exe (34.1 MB)
Downloading: 8192/35749888 bytes (0.0548864/35749888 bytes (1.5892928/35749888 bytes (2.51204224/35749888 bytes (3.4%1728512/35749888 bytes (4.8%2064384/35749888 bytes (5.8%2457600/35749888 bytes (6.9%2957312/35749888 bytes (8.3%3465216/35749888 bytes (9.7%3899392/35749888 bytes (10.94423680/35749888 bytes (12.44849664/35749888 bytes (13.65275648/35749888 bytes (14.85677056/35749888 bytes (15.96201344/35749888 bytes (17.36791168/35749888 bytes (19.07208960/35749888 bytes (20.27733248/35749888 bytes (21.68241152/35749888 bytes (23.18667136/35749888 bytes (24.29191424/35749888 bytes (25.79830400/35749888 bytes (27.510264576/35749888 bytes (28.7%10641408/35749888 bytes (29.8%11108352/35749888 bytes (31.1%11476992/35749888 bytes (32.1%11870208/35749888 bytes (33.2%12304384/35749888 bytes (

100%|███████████████████████████████████████████████████████████████████████████████| 728/728 [00:03<00:00, 184.72it/s]


[MoviePy] Done.
[MoviePy] Writing video //FS2.smpp.local\RTO\CIS-PD Videos\cisuabn14\cycle6_part1\NtsBts.mp4


100%|████████████████████████████████████████████████████████████████████████████████| 990/990 [00:35<00:00, 27.81it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: //FS2.smpp.local\RTO\CIS-PD Videos\cisuabn14\cycle6_part1\NtsBts.mp4 

